In [1]:
using Pkg
using Revise
Pkg.activate("..") 
Pkg.instantiate()
using JPEC, Plots
gr() 

  Activating project at `~/git/JPEC10/JPEC`


Plots.GRBackend()

In [2]:
using Revise
include("../src/DCON/fourfit.jl")
import .fourfit_make_metric, .MetricData, .fourfit_make_matrix, .MatrixData, .compute_eigenvalues

### Reading Eq

In [3]:
# 1. Define the input parameters for the equilibrium solver.
#    - eq_filename: The name of the g-file we just created.
#    - eq_type: "efit" for a standard g-file.
#    - jac_type: "boozer" or "hamada" for the output coordinates.
#    - mpsi, mtheta: Resolution of the output grid.
    equil_control = JPEC.Equilibrium.EquilibriumControl(;
    eq_filename="beta_1.00",        # eq_filename
    eq_type="efit",          # eq_type
    jac_type="hamada",        # jac_type
    grid_type="ldp",
    psilow=0.01,             # psilow
    psihigh=0.990)             # psihigh
equil_config = JPEC.Equilibrium.EquilibriumConfig(equil_control,JPEC.Equilibrium.EquilibriumOutput())
# 2. Run the main equilibrium setup function.
#    This will read the file, solve the direct problem, and return the final object.
println("Starting equilibrium reconstruction...")
plasma_eq = JPEC.Equilibrium.setup_equilibrium(equil_config)
#plasma_eq = JPEC.Equilibrium.setup_equilibrium("./DIIID_example/equil.toml")
println("Equilibrium reconstruction complete.")

┌ Info: Forcing hamada coordinate jacobian exponents: power_*
└ @ JPEC.Equilibrium /Users/seoda-eun/git/JPEC10/JPEC/src/Equilibrium/EquilibriumTypes.jl:48


Starting equilibrium reconstruction...
Equilibrium file: beta_1.00
--> Processing EFIT g-file: beta_1.00
--> Parsed from header: nw=129, nh=128
--> All main data blocks parsed successfully.
--> Creating 1D profile splines...
--> 1D Spline fitting complete.
--> Creating 2D psi spline...
--> 2D Spline fitting complete.
--- Starting Direct Equilibrium Processing ---
Finding magnetic axis...
  Iter  1: R = 3.115238, Z = 0.000008, |ΔR|=1.15e-01, |ΔZ|=1.11e-05
  Iter  2: R = 3.117635, Z = -0.000003, |ΔR|=2.40e-03, |ΔZ|=1.10e-05
  Iter  3: R = 3.117636, Z = -0.000003, |ΔR|=1.07e-06, |ΔZ|=1.04e-09
  Iter  4: R = 3.117636, Z = -0.000003, |ΔR|=2.12e-13, |ΔZ|=5.62e-16
Magnetic axis found at R=3.1176357129137466, Z=-2.9184696943092166e-6.
Finding inboard separatrix crossing...
  Restart attempt 1/6 with initial R = 2.675802
inboard separatrix found at R=1.5000175239830411.
Finding outboard separatrix crossing...
  Restart attempt 1/6 with initial R = 3.500684
outboard separatrix found at R=4.49999

In [ ]:
fieldnames(typeof(plasma_eq.rzphi))

### CAlc. Metric

In [ ]:
include("../src/DCON/fourfit.jl")
import .fourfit_make_metric, .MetricData, .fourfit_make_matrix, .MatrixData, .compute_eigenvalues

In [ ]:
# Fourfit Metric Calculation Example
println("📚 Fourfit Metric Calculation")
println("="^30)

using .FourfitMetric

try
    # Metric tensor calculation
    metric_result = FourfitMetric.fourfit_make_metric(
        plasma_eq.rzphi,  # 2D geometry spline
        plasma_eq.sq;     # 1D profile spline
        mpsi=128,          # Radial grid points
        mtheta=256,        # Poloidal grid points  
        mband=10,          # Fourier bandwidth
        fft_flag=true,    # Use FFT
        verbose=true      # Detailed output
    )
    
    println("✅ Metric calculation complete!")
    println("   Grid size: $(size(metric_result.fs))")
    



    
    # Evaluate at a specific point (if spline is available)
    if metric_result.fspline !== nothing
        psi_test = 0.5
        theta_test = 0.5
        
        components = FourfitMetric.fspline_eval_metric(metric_result, psi_test, theta_test)
        println("   (ψ=$psi_test, θ=$theta_test):")
        println("   g11 = $(round(components[1], digits=6))")
        println("   g22 = $(round(components[2], digits=6))")
        println("   g33 = $(round(components[3], digits=6))")
        println("   g23 = $(round(components[4], digits=6))")
        println("   g31 = $(round(components[5], digits=6))")
        println("   g12 = $(round(components[6], digits=6))")

        println("   jac = $(round(components[7], digits=6))")
        println("   jac1 = $(round(components[8], digits=6))")

    end
    
catch e
    println("❌Error: $e")
end

In [4]:
include("../src/DCON/fourfit.jl")
import .fourfit_make_metric, .MetricData, .fourfit_make_matrix, .MatrixData, .compute_eigenvalues

### Calc. Matric

In [129]:
using Revise

In [136]:
include("../src/DCON/fourfit.jl")
import .fourfit_make_metric, .MetricData, .fourfit_make_matrix, .MatrixData, .compute_eigenvalues

In [138]:
# Fourfit Matrix Calculation - Complete Example
println("🚀 Fourfit Matrix Calculation")
println("="^40)

using .FourfitMetric
matrix_result = nothing
try
    # Step 1: Matrix calculation with correct arguments
    println("2️⃣  Calculating MHD Coefficient Matrix...")
    
    matrix_result = FourfitMetric.fourfit_make_matrix(
        metric_result,       # metric tensor data
        plasma_eq.sq,        # 1D profile spline
        plasma_eq.rzphi,     # 2D geometry spline
        1.367;               # psio (toroidal flux normalization)
        nn=1,                # toroidal mode number
        mlow=-4,             # minimum poloidal mode
        mhigh=4,             # maximum poloidal mode
        power_flag=false,
        feval_flag=false,
        sas_flag=true,
        verbose=true
    )
    
    # Step 2: Analyze results
    println("3️⃣  Analyzing Results:")
    println("   Matrix storage sizes:")
    println("      F-matrix: $(size(matrix_result.fmats))")
    println("      G-matrix: $(size(matrix_result.gmats))")
    println("      K-matrix: $(size(matrix_result.kmats))")
    println("   Mode range: $(matrix_result.mlow) to $(matrix_result.mhigh)")
    println("   Number of modes: $(matrix_result.mpert)")
    
    println("✅ Matrix calculation completed successfully!")
    
catch e
    println("❌ Error occurred: $e")
    println("🔍 Error type: $(typeof(e))")
end

🚀 Fourfit Matrix Calculation
2️⃣  Calculating MHD Coefficient Matrix...
🚀 Fourfit Matrix Calculation - Complete Implementation
📊 Configuration:
   Mode range: -4 to 4 (total: 9 modes)
   Toroidal mode: n = 1
   Psi surfaces: 0 to 128
   Metric data size: (129, 88)

1️⃣  Allocating matrix storage...
   ✅ Matrix dimensions: 9 × 9
   ✅ Hermitian storage size: 44
   ✅ K-matrix storage size: 189

2️⃣  Computing matrices on each flux surface...
   Progress: 0.0% (ipsi = 0/128, ψ = 0.01)
   Progress: 9.4% (ipsi = 12/128, ψ = 0.0311)
   Progress: 18.8% (ipsi = 24/128, ψ = 0.0926)
   Progress: 28.1% (ipsi = 36/128, ψ = 0.1891)
   Progress: 37.5% (ipsi = 48/128, ψ = 0.3125)
   Progress: 46.9% (ipsi = 60/128, ψ = 0.452)
   Progress: 56.2% (ipsi = 72/128, ψ = 0.5956)
   Progress: 65.6% (ipsi = 84/128, ψ = 0.731)
   Progress: 75.0% (ipsi = 96/128, ψ = 0.8465)
   Progress: 84.4% (ipsi = 108/128, ψ = 0.9321)
   Progress: 93.8% (ipsi = 120/128, ψ = 0.9806)

3️⃣  Setting power factors...

4️⃣  Creating

In [140]:
matrix_result.amats[129,:]

81-element Vector{ComplexF64}:
  1267.9511758311253 + 0.0im
   781.8219401947784 + 0.010872870507432663im
   303.9129261098692 + 0.004457859531417795im
   57.47270599745727 + 1.5874875916168716e-5im
  23.644106605857143 + 0.00024090162093240368im
 -6.2572230834070375 + 0.0007055622785557876im
  3.4465632415722913 + 0.00016856820633988763im
   -3.04387131269166 - 0.0005262168773260617im
  1.9001778918391987 + 0.001442705528476617im
   781.8219401947784 - 0.010872870507432663im
                     ⋮
  1.9001778918391987 - 0.001442705528476617im
  -2.901500653045717 + 0.0006959579424019048im
    4.13821783544556 - 9.75853108454923e-5im
  -7.487201102957555 + 0.0001290070137160016im
  11.521964251865079 + 0.0011098357911283655im
 -20.026481648919223 - 0.0014565795158203165im
  46.769366224856114 + 0.0001810906113322885im
 -159.93439392183834 + 0.001386856846307313im
  360.60060804644974 + 0.0im

get_matrices

In [84]:
amat

11×11 Matrix{ComplexF64}:
  1697.18+0.0im            1048.1-0.014754im     …   0.84371-0.000365171im
   1048.1+0.014754im      1267.95+0.0im             -1.21073+0.0016493im
  383.492+0.00589584im    781.822+0.0108729im        1.97362-0.00153887im
  84.3563-0.000654576im   303.913+0.00445786im      -2.79399+0.000908706im
  28.5293+0.000684876im   57.4727+1.58749e-5im       4.35696-0.000182036im
 -5.64627+0.0012053im     23.6441+0.000240902im  …  -7.69623+0.000185653im
  2.97365+0.000324002im  -6.25722+0.000705562im      10.3461+0.00134123im
  -3.2211-0.000399483im   3.44656+0.000168568im     -24.1425-0.00136241im
  1.80623+0.00131177im   -3.04387-0.000526217im      40.6338+0.000289431im
 -1.21483-0.0014717im     1.90018+0.00144271im      -162.726+0.00100851im
  0.84371+0.000365171im  -1.21073-0.0016493im    …   562.994+0.0im

In [86]:
reader = ImatsReader("/Users/seoda-eun/Downloads/imats.xlsx")


ImatsReader(nψ=129, mpert=9, path="imats.xlsx")

In [114]:
emat./reader.E[129,:,:]

9×9 Matrix{ComplexF64}:
  -0.118945-0.0422004im   …   0.00184771+0.00154368im
   0.110639-0.00936398im     -0.00199582-0.0024498im
   -0.19425+1.02247im         0.00453243+0.00651789im
   0.200172-0.634382im        0.00391739+0.00632055im
  -0.238284+0.319389im        -0.0187432-0.0266332im
   0.113082-0.200648im    …    0.0451592+0.132577im
 -0.0208233+0.0476807im       -0.0680512-0.377339im
 -0.0760364+0.163342im          0.149316+0.0131044im
    1.06312+0.383654im         -0.132691-0.000533044im

In [103]:
reader.B[129,:,:]

9×9 Matrix{ComplexF64}:
          0.0+444.795im  -0.00289458+221.132im  …    0.0219038-2.62937im
 -0.000947784+221.132im          0.0+326.355im      -0.0185437+3.97792im
   -0.0224506-62.6758im   0.00289458+221.132im       0.0368239-5.68721im
   0.00211571+28.051im    -0.0192685-62.6773im      -0.0323863+10.2128im
     -0.00725-15.5791im  0.000835468+28.0522im      0.00779263-15.5317im
    0.0285525+10.2662im  -0.00731782-15.5732im  …   0.00812624+28.0606im
   -0.0236906-5.68057im    0.0290317+10.2595im     -0.00300596-62.6879im
    0.0326906+3.97923im   -0.0253323-5.6814im       -0.0297911+221.131im
  -0.00819469-2.63033im    0.0309223+3.97907im             0.0-502.726im

In [50]:
using FFTW

### Redaing imats(original DCON output)

In [144]:
# ────────────────────────────────────────────────────────────────
# Internal constants - Excel column positions (1-based)
const _COLS = Dict(
    :A => (4, 5),
    :B => (6, 7),
    :C => (8, 9),
    :D => (10, 11),
    :E => (12, 13),
    :H => (14, 15),
)
# ────────────────────────────────────────────────────────────────
mutable struct ImatsReader
    path      :: String
    psi       :: Vector{Float64}
    m_values  :: Vector{Int}
    npsi      :: Int
    mpert     :: Int
    A         :: Array{ComplexF64,3}
    B         :: Array{ComplexF64,3}
    C         :: Array{ComplexF64,3}
    D         :: Array{ComplexF64,3}
    E         :: Array{ComplexF64,3}
    H         :: Array{ComplexF64,3}
end

function ImatsReader(path::AbstractString)
    isfile(path) || throw(ArgumentError("File not found: $path"))
    # Temporary storage for variables
    temp_storage = []
    # 1) Open Excel file and read all data rows temporarily
    XLSX.openxlsx(path) do xf
        sheet = xf[1]
        # Use XLSX.eachtablerow to iterate through rows
        for row in XLSX.eachtablerow(sheet)
            # Skip if first column value is not a number (header or empty row)
            ψ = row[1]
            isa(ψ, Number) || continue
            # Read mi, mj and other values from the row
            mi = Int(row[2])
            mj = Int(row[3])
            # Store entire row data (columns containing real/imaginary parts of A~H)
            # Read data up to 15th column
            row_data = [row[c] for c in 1:15]
            push!(temp_storage, (ψ, mi, mj, row_data))
        end
    end # End of XLSX.openxlsx do block
    isempty(temp_storage) && error("No numeric data rows found in $path")
    
    # 2) Organize ψ / m lists
    psi_vals = sort(unique(getindex.(temp_storage, 1)))
    m_vals_all = vcat(getindex.(temp_storage, 2), getindex.(temp_storage, 3))
    m_vals   = sort(unique(m_vals_all))
    nψ       = length(psi_vals)
    mpert    = length(m_vals)
    
    # Create dictionaries for value -> index conversion
    ψidx = Dict(v => i for (i, v) in enumerate(psi_vals))
    midx = Dict(v => i for (i, v) in enumerate(m_vals))
    
    # 3) Initialize empty arrays to store final matrices
    shp = (nψ, mpert, mpert)
    mats = Dict(name => zeros(ComplexF64, shp) for name in keys(_COLS))
    
    # 4) Fill arrays with temporarily stored data
    for (ψ, mi, mj, row_cells) in temp_storage
        k = ψidx[ψ]
        i = midx[mi]
        j = midx[mj]
        for (name, (recol, imcol)) in _COLS
            # row_cells has 1-based indexing
            real_val = row_cells[recol]
            imag_val = row_cells[imcol]
            # Handle empty cells (nothing) or non-numeric values as 0.0
            re = isa(real_val, Number) ? Float64(real_val) : 0.0
            im = isa(imag_val, Number) ? Float64(imag_val) : 0.0
            mats[name][k, i, j] = ComplexF64(re, im)
        end
    end
    
    # Create and return final object
    return ImatsReader(String(path), psi_vals, m_vals, nψ, mpert,
               mats[:A], mats[:B], mats[:C], mats[:D], mats[:E], mats[:H])
end

# ────────────────────────────────────────────────────────────────
# Internal: ψ index interpretation
function _ψindex(r::ImatsReader; ψ_index::Union{Nothing,Int}=nothing,
                                   ψ_value::Union{Nothing,Real}=nothing)
    ψ_index !== nothing && return ψ_index
    ψ_value !== nothing || error("Either ψ_index or ψ_value must be specified")
    return argmin(abs.(r.psi .- ψ_value))
end

# ────────────────────────────────────────────────────────────────
"Return single matrix: name ∈ (:A,:B,:C,:D,:E,:H)"
function get_matrix(r::ImatsReader, name::Symbol;
                    ψ_index::Union{Nothing,Int}=nothing,
                    ψ_value::Union{Nothing,Real}=nothing)
    name = Symbol(uppercase(String(name)))
    hasproperty(r, name) || error("unknown matrix $name")
    k = _ψindex(r; ψ_index, ψ_value)
    return getfield(r, name)[k, :, :]
end

"Return one ψ cross-section (6 matrices) as Dict"
function get_matrices(r::ImatsReader; ψ_index::Union{Nothing,Int}=nothing,
                                         ψ_value::Union{Nothing,Real}=nothing)
    k = _ψindex(r; ψ_index, ψ_value)
    return Dict(name => getfield(r, name)[k, :, :] for name in (:A,:B,:C,:D,:E,:H))
end

get_matrices

In [146]:
read=ImatsReader("/Users/seoda-eun/Downloads/imats.xlsx")

ImatsReader(nψ=129, mpert=9, path="imats.xlsx")

In [147]:
read.A[end,:,:]

9×9 Matrix{ComplexF64}:
   1267.5+0.0im          782.052+0.0595154im  …   1.91602+0.00606395im
  782.052-0.0595154im    908.889+0.0im           -2.92174-0.0226763im
  304.578-0.0522503im    548.816-0.0442843im      4.13194+0.0194073im
  57.8667-0.0157114im    235.391-0.0418065im     -7.47055-0.0216927im
  23.7925-0.0309502im    34.6922-0.0110717im      11.5202+0.00579372im
 -6.25002+0.00870424im   19.6032-0.0223199im  …  -19.9741+0.00320115im
  3.43404-0.00853435im  -6.75865+0.0139202im      46.8702+0.00275821im
 -3.06309+0.024506im     3.82547-0.0143416im     -160.067-0.018649im
  1.91602-0.00606395im  -2.92174+0.0226763im      359.946+0.0im

In [ ]:
reshape(matrix_result.amats[129, :], 9, 9)./read.A[end,:,:]

9×9 Matrix{ComplexF64}:
  1.00036+0.0im          0.999706-8.99822e-5im   …  0.991719-0.00389162im
 0.999706+8.99822e-5im    1.00001+0.0im             0.993011-0.0079452im
 0.997816+0.000185811im  0.999381+9.43505e-5im        1.0015-0.00472755im
 0.993191+0.000269936im  0.997599+0.000190814im      1.00222-0.00292748im
 0.993759+0.00130284im   0.991106+0.000332411im      1.00016-0.00040666im
  1.00115+0.00128139im   0.994771+0.00112444im   …   1.00262+0.000233608im
  1.00364+0.00254335im    1.00133+0.00201552im      0.997848-5.48575e-5im
 0.993661+0.00812152im    1.00241+0.00378235im      0.999171-0.000125075im
 0.991719+0.00389162im   0.993011+0.0079452im        1.00182+0.0im